In [3]:
from pathlib import Path
import pandas as pd
import os
import shutil
import numpy as np
from pathlib import Path

In [4]:
CELLMAPS = "~/.venv/lib/python3.8/site-packages"
DATA_FOLDER = "/data/ishang/music/"
INPUT_FOLDER = "./examples/"
feature_path = Path("/data/ishang/music/cell_features_test_default_cell_v1.npz")
cell_id_path = Path("/data/ishang/music/test.csv")
# hpa_images_path = Path("/data/ishang/music/cells_publicHPA_v23.csv")
labels = Path("/data/ishang/music/cells_publicHPA.csv")
hpa_images_path = Path("/data/ishang/music/IF-image.csv")

# Example Dir

In [3]:
# to run the pipeline end to end, you need your own version of the examples directory
examples_dir = Path("./examples")
baitlist = examples_dir / "baitlist.tsv"
edgelist = examples_dir / "edgelist.tsv"
samples = examples_dir / "samples.csv"
unique = examples_dir / "unique.csv"

bait_df = pd.read_csv(baitlist, sep="\t")
edge_df = pd.read_csv(edgelist, sep="\t")
sample_df = pd.read_csv(samples)
unique_df = pd.read_csv(unique)

In [4]:
bait_df.head()

,GeneSymbol,GeneID,# Interactors
0,PIK3CA,101928739,2783


In [14]:
edge_df.head()

,GeneID1,Symbol1,GeneID2,Symbol2
0,101928739,PIK3CA,219541,MED19
1,101928739,PIK3CA,26030,PLEKHG3
2,101928739,PIK3CA,129446,XIRP2
3,101928739,PIK3CA,644815,FAM83G
4,101928739,PIK3CA,23347,SMCHD1


In [15]:
sample_df.head()

,filename,if_plate_id,position,sample,status,locations,antibody,ensembl_ids,gene_names
0,/archive/7/7_C5_1_,7,C5,1,35,"Cytosol,Nuclear speckles",HPA005910,ENSG00000011007,ELOA
1,/archive/7/7_C5_2_,7,C5,2,35,"Cytosol,Nuclear speckles",HPA005910,ENSG00000011007,ELOA
2,/archive/7/7_E8_1_,7,E8,1,35,Nuclear speckles,HPA006628,ENSG00000239306,RBM14
3,/archive/7/7_E8_2_,7,E8,2,35,Nuclear speckles,HPA006628,ENSG00000239306,RBM14
4,/archive/10/10_G9_1_,10,G9,1,35,Nuclear speckles,HPA008762,ENSG00000116754,SRSF11


In [30]:
# looks like this is just a row for an antibody/ensembl_id pair, but doesn't have all the antibodies for an id
unique_df.head()

,antibody,ensembl_ids,gene_names,atlas_name,locations,n_location
0,CAB004303,ENSG00000072110,ACTN1,U-2 OS,"Focal adhesion sites,Plasma membrane",2
1,HPA046964,ENSG00000182718,ANXA2,U-2 OS,"Cytosol,Plasma membrane",2
2,HPA002564,ENSG00000100823,APEX1,U-2 OS,Nucleoplasm,1
3,HPA006550,ENSG00000111229,ARPC3,U-2 OS,Nucleoplasm,1
4,HPA042816,ENSG00000204256,BRD2,U-2 OS,Nuclear speckles,1


In [32]:
assert set(sample_df["ensembl_ids"]) == set(unique_df["ensembl_ids"])
print(len(sample_df), len(unique_df))
print(set(unique_df["atlas_name"]))
print(unique_df[unique_df["ensembl_ids"] == "ENSG00000100823"])
sample_df[sample_df["ensembl_ids"] == "ENSG00000100823"]

483 168
{'U-2 OS'}
    antibody      ensembl_ids gene_names atlas_name    locations  n_location
2  HPA002564  ENSG00000100823      APEX1     U-2 OS  Nucleoplasm           1


,filename,if_plate_id,position,sample,status,locations,antibody,ensembl_ids,gene_names
8,/archive/17/17_G2_1_,17,G2,1,35,Nucleoplasm,HPA002564,ENSG00000100823,APEX1
9,/archive/17/17_G2_2_,17,G2,2,35,Nucleoplasm,HPA002564,ENSG00000100823,APEX1
44,/archive/81/81_D2_1_,81,D2,1,35,Nucleoplasm,HPA000956,ENSG00000100823,APEX1
45,/archive/81/81_D2_2_,81,D2,2,35,Nucleoplasm,HPA000956,ENSG00000100823,APEX1
232,/archive/645/645_A1_1_,645,A1,1,35,Nucleoplasm,CAB047307,ENSG00000100823,APEX1
233,/archive/645/645_A1_3_,645,A1,3,35,Nucleoplasm,CAB047307,ENSG00000100823,APEX1
240,/archive/662/662_B11_1_,662,B11,1,35,Nucleoplasm,CAB004294,ENSG00000100823,APEX1
241,/archive/662/662_B11_2_,662,B11,2,35,Nucleoplasm,CAB004294,ENSG00000100823,APEX1


In [49]:
# indexed by ensembl_ids, collect the list of antibodies and list of locations
ensemble_ids = unique_df["ensembl_ids"].unique()
collapsed_rows = {id: {"antibodies": set(), "locations": set()} for id in ensemble_ids}

def parse_row(df_row, df_cols):
    return {df_cols[i]: df_row[1][i] for i in range(len(df_cols))}

for row in sample_df.iterrows():
    row_dict = parse_row(row, sample_df.columns)
    ensembl_id = row_dict["ensembl_ids"]
    antibody = row_dict["antibody"]
    if type(row_dict["locations"]) == float:
        continue
    locations = row_dict["locations"].split(",")
    collapsed_rows[ensembl_id]["antibodies"].add(antibody)
    collapsed_rows[ensembl_id]["locations"].update(locations)
ensembl_view_df = pd.DataFrame.from_dict(collapsed_rows, orient="index")
ensembl_view_df.head()

,antibodies,locations
ENSG00000072110,{CAB004303},"{Plasma membrane, Focal adhesion sites}"
ENSG00000182718,{HPA046964},"{Plasma membrane, Cytosol}"
ENSG00000100823,"{HPA002564, CAB047307, HPA000956, CAB004294}",{Nucleoplasm}
ENSG00000111229,{HPA006550},{Nucleoplasm}
ENSG00000204256,{HPA042816},{Nuclear speckles}


In [4]:
# get list of protein+antibody images from HPA
# get list of all proteins from bioplex
# get number of proteins with each localization in HPA and then pick one to prototype the rest with
# have a list that is the one used in the MuSIC paper
# do a robustness test (just changing antibodies randomly, maybe 100 times)
# make a hierarchy with all antibodies and get IoU of all clusters <= cutoff size
# enrich for CCD in both
# then use df to partition and drop out in that phase if bigger

In [3]:
# get csv of HPA images from the /data folder
if_images_csv = Path("~/HPA-embedding/ifimages_v23.csv")
hpa_df = pd.read_csv(if_images_csv)
print(hpa_df.columns)
hpa_df.head()

Index(['filename', 'jpg_prefix', 'if_plate_id', 'position', 'sample', 'status',
       'Image status name', 'locations', 'staining characteristics',
       'unspecific', 'antibody', 'ensembl_ids', 'gene_names', 'atlas_name',
       'versions', 'earliest_version', 'first_released', 'latest_version',
       'Spatial cell cycle', 'Intensity cell cycle', 'Annotated cell cycle',
       'gain', 'x_pos', 'y_pos', 'z_pos', 'Experiment state', 'Ab state',
       'Max tpm', 'Finished in genes', 'Protocol',
       'Gene reliability (in release)', 'Gene reliability (lims)',
       'Cell count', 'well_location_predictions_all'],
      dtype='object')


,filename,jpg_prefix,if_plate_id,position,sample,status,Image status name,locations,staining characteristics,unspecific,...,z_pos,Experiment state,Ab state,Max tpm,Finished in genes,Protocol,Gene reliability (in release),Gene reliability (lims),Cell count,well_location_predictions_all
0,/archive/1/1_A1_1_,https://lims.proteinatlas.org/images/992/1_A1_1_,1,A1,1,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,7.0,NaN
1,/archive/1/1_A1_2_,https://lims.proteinatlas.org/images/992/1_A1_2_,1,A1,2,35,Annotated / Proteinatlas,Golgi apparatus,NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,16.13,ENSG00000066455,PFA,Supported,Supported,6.0,NaN
2,/archive/1/1_A3_1_,https://lims.proteinatlas.org/images/2899/1_A3_1_,1,A3,1,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,2.74,ENSG00000183092,PFA,Approved,Approved,4.0,NaN
3,/archive/1/1_A3_2_,https://lims.proteinatlas.org/images/2899/1_A3_2_,1,A3,2,35,Annotated / Proteinatlas,"Cytosol,Nucleoplasm",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,2.74,ENSG00000183092,PFA,Approved,Approved,6.0,NaN
4,/archive/1/1_A6_1_,https://lims.proteinatlas.org/images/609/1_A6_1_,1,A6,1,35,Annotated / Proteinatlas,"Endoplasmic reticulum,Nuclear membrane",NaN,0.0,...,NaN,IF_FINISHED,IF_FINISHED,81.38,ENSG00000102119,PFA,Enhanced,Enhanced,5.0,NaN


# Load SC Embeddings

In [5]:
CELLMAPS = "~/.venv/lib/python3.8/site-packages"
DATA_FOLDER = "/data/ishang/music/"
INPUT_FOLDER = "./examples/"
feature_path = Path("/data/ishang/music/cell_features_test_default_cell_v1.npz")
cell_id_path = Path("/data/ishang/music/test.csv")
# hpa_images_path = Path("/data/ishang/music/cells_publicHPA_v23.csv")
labels = Path("/data/ishang/music/cells_publicHPA.csv")
hpa_images_path = Path("/data/ishang/music/IF-image.csv")

In [6]:
cell_id_df = pd.read_csv(cell_id_path)
labels_df = pd.read_csv(labels)
hpa_images_df = pd.read_csv(hpa_images_path)
sc_image_features = np.load(feature_path)

/tmp/ipykernel_3221317/3684130873.py:3: DtypeWarning: Columns (7,8,14,16,18,19,33) have mixed types. Specify dtype option on import or set low_memory=False.
  hpa_images_df = pd.read_csv(hpa_images_path)


# select genes to include

In [5]:
from pathlib import Path
import shutil
import pandas as pd
import numpy as np

In [6]:
img_dir = Path(outdir_image_embedding)
img_dir_filtered = Path('./2.image_embedding_filtered')
if img_dir_filtered.exists():
    shutil.rmtree(img_dir_filtered)

ppi_dir = Path(outdir_ppi_download)
ppi_dir_filtered = Path('./1.ppi_download_filtered')
if ppi_dir_filtered.exists():
    shutil.rmtree(ppi_dir_filtered)

shutil.copytree(img_dir, img_dir_filtered)
shutil.copytree(ppi_dir, ppi_dir_filtered)

PosixPath('1.ppi_download_filtered')

In [22]:
ppi_edgelist = ppi_dir_filtered / 'ppi_edgelist.tsv'
ppi_df = pd.read_csv(ppi_edgelist, sep='\t')
ppi_df.head()

,geneA,geneB
0,PIK3CA-DT,MED19
1,PIK3CA-DT,PLEKHG3
2,PIK3CA-DT,XIRP2
3,PIK3CA-DT,FAM83G
4,PIK3CA-DT,SMCHD1


In [23]:
img_emd = img_dir_filtered / 'image_emd.tsv'
img_df = pd.read_csv(img_emd, sep='\t')
img_df.head()

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
SREK1,0.028495,-0.131659,-0.140246,-0.108417,-0.100102,-0.134292,-0.042231,-0.060445,0.130666,-0.034349,...,-0.129697,0.237701,0.023639,0.173883,-0.078425,-0.094233,0.140823,0.031387,0.437817,-0.140267
TRIP12,0.120922,-0.131659,0.431988,0.099261,-0.100102,-0.134292,0.022078,-0.060445,0.003795,-0.147992,...,-0.129697,-0.013884,0.023639,0.502489,-0.078425,0.053833,0.134783,0.031387,0.304909,0.177448
MYO1B,0.022881,0.133529,-0.140246,-0.034903,-0.098910,-0.134292,0.150110,0.225479,-0.138573,0.048582,...,-0.129697,-0.013884,0.570602,-0.046523,-0.078425,0.052769,0.011603,0.031387,-0.029221,-0.140267
SRSF11,0.022881,-0.131659,0.027396,-0.108417,-0.100102,-0.134292,0.027179,-0.060445,0.151627,-0.033765,...,-0.129697,0.124886,0.023639,0.416759,-0.078425,-0.090855,0.029354,0.031387,0.667623,0.043594
FAM204A,0.022881,-0.131659,-0.140246,-0.108417,-0.073226,-0.134292,-0.042231,-0.060445,-0.138573,-0.147992,...,0.523106,0.112435,0.526269,-0.046523,0.039800,0.024477,0.011603,0.317094,0.254935,-0.140267


In [24]:
ppi_genes = set(ppi_df['geneA']) | set(ppi_df['geneB'])
img_genes = set(img_df.index)
print('Number of genes in PPI:', len(ppi_genes))
print('Number of genes in image embedding:', len(img_genes))
assert (ppi_genes & img_genes) == img_genes

gene_sample = np.random.choice(list(img_genes), int(0.9 * len(img_genes)), replace=False)
gene_sample = set(gene_sample)
gene_sample.add(ppi_df['geneA'].iloc[0])
gene_sample.add(ppi_df['geneB'].iloc[0]) # to make sure we got at least one on each side
gene_sample = list(gene_sample)
print(len(gene_sample))

Number of genes in PPI: 1010
Number of genes in image embedding: 167
152


In [25]:
print(len(ppi_df), len(img_df))
ppi_df = ppi_df[ppi_df['geneA'].isin(gene_sample) & ppi_df['geneB'].isin(gene_sample)]
gene_sample = list(set(gene_sample) & set(img_df.index))
img_df = img_df.loc[gene_sample]
print(len(ppi_df), len(img_df))

ppi_df.to_csv(ppi_edgelist, sep='\t', index=False)
img_df.to_csv(img_emd, sep='\t', index=True)

2783 167
431 150


# ppi embedding

run node2vec on PPI network

Note: may want to test different p and q values

In [26]:
# outdir_ppi_embedding = './2.ppi_embedding'
outdir_ppi_embedding = './2.ppi_embedding_filtered'
inputdir = './1.ppi_download'

In [27]:
%run $CELLMAPS/cellmaps_ppi_embedding/cellmaps_ppi_embeddingcmd.py $outdir_ppi_embedding --inputdir $inputdir

/home/ishang/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating walks (CPU: 2): 100%|██████████| 2/2 [00:08<00:00,  4.10s/it]


# coembedding

runs implementation of MUSE to create a single integrated embedding for each protein 

note: may want to alter # epochs to reduce training time

In [28]:
outdir_coembedding = './3.coembedding'
ppi_embeddingdir = './2.ppi_embedding'
# image_embeddingdir = './2.image_embedding'
image_embeddingdir = './2.image_embedding_filtered'
image_downloaddir = './1.image_download'

In [30]:
%run $CELLMAPS/cellmaps_coembedding/cellmaps_coembeddingcmd.py $outdir_coembedding --ppi_embeddingdir $ppi_embeddingdir --image_embeddingdir $image_embeddingdir

Saving embedding: 0it [00:00, ?it/s]

Finding 10 nearest neighbors using cosine metric and 'brute' algorithm


/home/ishang/.venv/lib/python3.8/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Neighbors computed in 0.224273681640625 seconds
Jaccard graph constructed in 4.232062101364136 seconds
Wrote graph to binary file in 0.0009772777557373047 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.606904
After 8 runs, maximum modularity is Q = 0.608589
Louvain completed 28 runs in 1.105015516281128 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 10.134282350540161 seconds
Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 0.1885695457458496 seconds
Jaccard graph constructed in 5.251671314239502 seconds
Wrote graph to binary file in 0.0008897781372070312 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.795804
Louvain completed 21 runs in 0.7971978187561035 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 11.178891658782959 seconds


/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:35: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:42: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zer

...

/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:35: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:42: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zer

Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 0.16331815719604492 seconds
Jaccard graph constructed in 5.570333957672119 seconds
Wrote graph to binary file in 0.0009286403656005859 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.725984
After 2 runs, maximum modularity is Q = 0.727476
After 6 runs, maximum modularity is Q = 0.729313
After 9 runs, maximum modularity is Q = 0.730876
Louvain completed 29 runs in 1.8766648769378662 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 13.569924116134644 seconds
Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 0.1611795425415039 seconds
Jaccard graph constructed in 5.968934059143066 seconds
Wrote graph to binary file in 0.0009548664093017578 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.865522
Louvain completed 21 runs in 0.9707355499267578 seco

/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:35: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:42: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zer

Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 0.27766871452331543 seconds
Jaccard graph constructed in 5.500546216964722 seconds
Wrote graph to binary file in 0.0009944438934326172 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.720663
After 2 runs, maximum modularity is Q = 0.725088
Louvain completed 22 runs in 1.1231739521026611 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 12.440693616867065 seconds
Finding 10 nearest neighbors using cosine metric and 'brute' algorithm
Neighbors computed in 0.18557262420654297 seconds
Jaccard graph constructed in 5.672888278961182 seconds
Wrote graph to binary file in 0.0009174346923828125 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.866747
Louvain completed 21 runs in 0.9308044910430908 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 12.315386295318604 secon

/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:35: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  distinct_indices = torch.logical_and(torch.logical_and(i_not_equal_j, i_not_equal_k), j_not_equal_k)
/home/ishang/.venv/lib/python3.8/site-packages/cellmaps_coembedding/muse_sc/triplet_loss.py:42: UserWarning: An output with one or more elements was resized since it had shape [64, 64, 1], which does not match the required output shape [64, 64, 64]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zer

# generate hierarchy

Calculates similarities between embeddings then runs HiDeF community detection to make hierarchy
Note 

In [31]:
output_hierarchy = './4.hierarchy'
coembedding_dir = './3.coembedding'

In [34]:
%tb
%run $CELLMAPS/cellmaps_generate_hierarchy/cellmaps_generate_hierarchycmd.py $output_hierarchy --coembedding_dir $coembedding_dir

SystemExit: 2

SystemExit: 2

# eval

In [1]:
output_hierarchy = '/cellar/users/lvschaffer/Data/MuSIC/U2OS/coembedding/muse_pytorch/new_embeddings/U2OS_music_pipeline_analysis/cellmaps_toolkit/5.eval_hierarchy'
inputdir = '/cellar/users/lvschaffer/Data/MuSIC/U2OS/coembedding/muse_pytorch/new_embeddings/U2OS_music_pipeline_analysis/cellmaps_toolkit/4.hierarchy'

In [4]:
%run $CELLMAPS/cellmaps_hierarchyeval/cellmaps_hierarchyeval/cellmaps_hierarchyevalcmd.py $output_hierarchy --hierarchy_dir $inputdir

Exception: File `'/home/ishang/.venv/lib/python3.8/site-packages//cellmaps_hierarchyeval/cellmaps_hierarchyeval/cellmaps_hierarchyevalcmd.py'` not found.